In [2]:
import pandas as pd

In [3]:
# read in data
movie_df = pd.read_csv("output/clean_df.csv")
movie_df.head(5)

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating,boxoffice
0,0,Pirates of the Caribbean: Dead Man's Chest,PG-13,2006-07-07,151,"Action, Adventure, Fantasy",Gore Verbinski,"Ted Elliott, Terry Rossio, Stuart Beattie","Johnny Depp, Orlando Bloom, Keira Knightley","English, Turkish, Greek, Mandarin, French",https://m.media-amazon.com/images/M/MV5BMTcwOD...,53.0,7.3,423315812
1,1,Night at the Museum,PG,2006-12-22,108,"Adventure, Comedy, Family",Shawn Levy,"Robert Ben Garant, Thomas Lennon, Milan Trenc","Ben Stiller, Carla Gugino, Ricky Gervais","English, Italian, Hebrew",https://m.media-amazon.com/images/M/MV5BMTQyOT...,48.0,6.4,250863268
2,2,Cars,G,2006-06-09,117,"Animation, Adventure, Comedy","John Lasseter, Joe Ranft","John Lasseter, Joe Ranft, Jorgen Klubien","Owen Wilson, Bonnie Hunt, Paul Newman","English, Italian, Japanese, Yiddish",https://m.media-amazon.com/images/M/MV5BMTg5Nz...,73.0,7.1,244082982
3,3,X-Men: The Last Stand,PG-13,2006-05-26,104,"Action, Adventure, Sci-Fi",Brett Ratner,"Simon Kinberg, Zak Penn","Patrick Stewart, Hugh Jackman, Halle Berry",English,https://m.media-amazon.com/images/M/MV5BNDBhND...,58.0,6.7,234362462
4,4,The Da Vinci Code,PG-13,2006-05-19,149,"Mystery, Thriller",Ron Howard,"Akiva Goldsman, Dan Brown","Tom Hanks, Audrey Tautou, Jean Reno","English, French, Latin, Spanish",https://m.media-amazon.com/images/M/MV5BMjIxMj...,46.0,6.6,217536138


In [4]:
# create dfs for tables
role_df = pd.DataFrame({"role_id": [0,1,2], "role": ["Director", "Writer", "Actor"]})
person_df = pd.DataFrame({"person_id": [], "name": []})
movie_role_person_df = pd.DataFrame(columns=["movie", "role", "person"])

In [5]:
# set indexes
role_df.set_index(['role_id'], inplace=True)
person_df.set_index(["person_id"], inplace=True)
role_df

,role
role_id,
0,Director
1,Writer
2,Actor


In [6]:
person_df.head()

,name
person_id,


In [7]:
# iterate through movie_df
for index, row in movie_df.iterrows():
    # split out all people with roles
    directors = row["Director"].split(", ")
    writers = row["Writer"].split(", ")
    actors = row["Actors"].split(", ")

    for director in directors:
        # check if person in person_df
        if director not in person_df["name"].values:
            new_person_row = {"person": director}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 0, "person": person_df.index[person_df["name"] == director].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)
        
    for writer in writers:
        # check if person in person_df
        if writer not in person_df["name"].values:
            new_person_row = {"person": writer}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 1, "person": person_df.index[person_df["name"] == writer].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)

    for actor in actors:
        # check if person in person_df
        if actor not in person_df["name"].values:
            new_person_row = {"person": actor}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 2, "person": person_df.index[person_df["name"] == actor].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)
    


C:\Users\12164\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  # Remove the CWD from sys.path while we load stuff.
C:\Users\12164\anaconda3\envs\PythonData\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
C:\Users\12164\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
C:\Users\12164\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [8]:
person_df.head()

,name,person
0,NaN,Gore Verbinski
1,NaN,Ted Elliott
2,NaN,Terry Rossio
3,NaN,Stuart Beattie
4,NaN,Johnny Depp


In [9]:
# converts person column from list type to string type
movie_role_person_df['person']  = [','.join(map(str, row)) for row in movie_role_person_df['person']]

In [10]:
movie_role_person_df

,movie,role,person
0,0,0,
1,0,1,
2,0,1,
3,0,1,
4,0,2,
...,...,...,...
27112,6894,1,
27113,6894,1,
27114,6894,2,
27115,6894,2,


In [11]:
role_df.to_csv("output/role_table.csv")
person_df.to_csv("output/person_table.csv", index=False)
movie_role_person_df.to_csv("output/movie_role_person_junction_table.csv", index=False)